In [1]:
ENV["CLIMACOMMS_DEVICE"] = "CUDA"
ENV["CLIMA_NAME_CUDA_KERNELS_FROM_STACK_TRACE"] = "true"

"true"

In [2]:
pwd()

"/home/pbachant/calkit/clima-horizontal-diffusion"

In [3]:
import Pkg

Pkg.activate("./ClimaAtmos.jl/.buildkite")
Pkg.instantiate()
Pkg.precompile()
Pkg.status()

  Activating project at `~/calkit/clima-horizontal-diffusion/ClimaAtmos.jl/.buildkite`
Precompiling project...
  53528.4 ms  ✓ PrecompileCI
  1 dependency successfully precompiled in 63 seconds. 552 already precompiled.


Status `~/calkit/clima-horizontal-diffusion/ClimaAtmos.jl/.buildkite/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [a14bc488] ArtifactWrappers v0.2.0
  [6e4b80f9] BenchmarkTools v1.6.3
⌃ [052768ef] CUDA v5.8.5
  [13f3f980] CairoMakie v0.15.8
  [29b5916a] ClimaAnalysis v0.5.20
  [b2c96348] ClimaAtmos v0.32.0 `..`
  [3a4d1b5c] ClimaComms v0.6.10
⌃ [d414da3d] ClimaCore v0.14.43
  [c2caaa1d] ClimaCoreSpectra v0.1.4
  [d934ef94] ClimaCoreTempestRemap v0.3.18
⌅ [1ecacbb8] ClimaDiagnostics v0.2.15
  [e0c89595] ClimaReproducibilityTests v0.1.1
  [595c0a79] ClimaTimeSteppers v0.8.5
  [b3f4f4ca] ClimaUtilities v0.1.27
⌃ [2b5f629d] DiffEqBase v6.183.2
  [f67ccb44] HDF5 v0.17.2
⌅ [a98d9a8b] Interpolations v0.15.1
  [8197267c] IntervalSets v0.7.13
⌅ [c3a54625] JET v0.9.20
  [033835bb] JLD2 v0.6.3
  [682c06a0] JSON v1.3.0
  [ba0b0d4f] Krylov v0.10.3
  [da04e1cc] MPI v0.20.23
⌃ [85f8d34a] NCDatasets v0.14.8
  [0d71be07] NullBroadcasts v0.1.0
  [bac558e1] OrderedCollections v1.8.1
  [76d61242] Precompi

In [4]:
redirect_stderr(IOContext(stderr, :stacktrace_types_limited => Ref(false)))
import ClimaComms
ClimaComms.@import_required_backends
import Random
Random.seed!(1234)
import ClimaAtmos as CA
import CUDA
import JLD2


for config_file in ["config/base.yml", "config/smag.yml"]
    if !(config_file in ARGS)
        append!(ARGS, ["--config_file", config_file])
    end
end

include("./ClimaAtmos.jl/perf/common.jl")
(; config_file, job_id) = CA.commandline_kwargs()
config = CA.AtmosConfig(config_file; job_id)

simulation = CA.get_simulation(config)
(; integrator) = simulation;

# Snapshot the current state/params/time
Y = deepcopy(integrator.u)
Yt = deepcopy(integrator.u)
Yt .= 0
p = integrator.p
t = integrator.t

# Resolve the Smagorinsky–Lilly model from the cached atmos physics
model = p.atmos.smagorinsky_lilly
@assert model isa CA.SmagorinskyLilly "Could not find SmagorinskyLilly model; adjust the lookup above."

[ Info: Precompiling ClimaAtmos [b2c96348-7fb7-4fe0-8da9-78d88439e717] (cache misses: wrong dep version loaded (18))
┌ Warning: Failed to convert `config_dict["vert_diff"] = false` to default type String, keeping original value
└ @ ClimaAtmos ~/calkit/clima-horizontal-diffusion/ClimaAtmos.jl/src/solver/yaml_helper.jl:78
┌ Info: Making AtmosConfig with config files: 
│    /home/pbachant/calkit/clima-horizontal-diffusion/ClimaAtmos.jl/src/solver/../../config/default_configs/default_config.yml
│    config/base.yml
└    config/smag.yml
[ Info: Running on CUDADevice
[ Info: Setting up single-process ClimaAtmos run
┌ Info: Time info:
│   dt = "30secs"
│   t_start = "0secs"
│   t_end = "3days"
└   floor_n_steps = 8640
┌ Info: numerics 
│        `energy_q_tot_upwinding`::`Val{:vanleer_limiter}`
│              `tracer_upwinding`::`Val{:first_order}`
│     `edmfx_mse_q_tot_upwinding`::`Val{:first_order}`
│       `edmfx_sgsflux_upwinding`::`Val{:none}`
│        `edmfx_tracer_upwinding`::`Val{:fir

In [5]:
# Precompute Smagorinsky–Lilly quantities
CA.set_smagorinsky_lilly_precomputed_quantities!(Y, p, model)
bl_p = CUDA.@elapsed CA.set_smagorinsky_lilly_precomputed_quantities!(Y, p, model)
bl_p

0.001503552f0

In [27]:
# Compute the diffusion tendency
CA.horizontal_smagorinsky_lilly_tendency!(Yt, Y, p, t, model)
bl = CUDA.@elapsed CA.horizontal_smagorinsky_lilly_tendency!(Yt, Y, p, t, model)
bl

0.001017824f0

In [53]:
using ClimaAtmos
import ClimaAtmos: C12, C3, wdivₕ, gradₕ, ᶠinterp, ᶜgradᵥ, specific, @name,
    SmagorinskyLilly, foreach_gs_tracer, is_smagorinsky_horizontal, lazy
import ClimaAtmos.Thermodynamics as TD
import ClimaAtmos.Parameters as CAP

function horizontal_smagorinsky_lilly_tendency_mod!(Yₜ, Y, p, t, model::SmagorinskyLilly)
    is_smagorinsky_horizontal(model) || return nothing
    (; ᶜts, ᶜS, ᶠS, ᶜνₜ_h, ᶜD_h) = p.precomputed
    (; ᶠtemp_UVWxUVW, ᶠtemp_scalar) = p.scratch
    thermo_params = CAP.thermodynamics_params(p.params)
    ᶜρ = Y.c.ρ
    ᶠρ = @. ᶠtemp_scalar = ᶠinterp(ᶜρ)

    # Subgrid-scale momentum flux tensor, `τ = -2 νₜ ∘ S`
    ᶠνₜ_h = @. lazy(ᶠinterp(ᶜνₜ_h))
    ᶜτ_smag = @. lazy(-2 * ᶜνₜ_h * ᶜS) # This was lazified
    ᶠτ_smag = @. ᶠtemp_UVWxUVW = -2 * ᶠνₜ_h * ᶠS # TODO: Lazify once we can mix lazy horizontal & vertical operations

    # Apply to tendencies
    ## Horizontal momentum tendency
    @. Yₜ.c.uₕ -= C12(wdivₕ(ᶜρ * ᶜτ_smag) / ᶜρ)
    ## Vertical momentum tendency
    @. Yₜ.f.u₃ -= C3(wdivₕ(ᶠρ * ᶠτ_smag) / ᶠρ)

    ## Total energy tendency
    ᶜe_tot = @. lazy(specific(Y.c.ρe_tot, ᶜρ))
    ᶜh_tot = @. lazy(TD.total_specific_enthalpy(thermo_params, ᶜts, ᶜe_tot))
    @. Yₜ.c.ρe_tot += wdivₕ(ᶜρ * ᶜD_h * gradₕ(ᶜh_tot))

    ## Tracer diffusion and associated mass changes
    foreach_gs_tracer(Yₜ, Y) do ᶜρχₜ, ᶜρχ, ρχ_name
        ᶜχ = @. lazy(specific(ᶜρχ, ᶜρ))
        ᶜ∇ₕρD∇χₜ = @. lazy(wdivₕ(ᶜρ * ᶜD_h * gradₕ(ᶜχ)))
        @. ᶜρχₜ += ᶜ∇ₕρD∇χₜ
        # Rain and snow does not affect the mass
        if ρχ_name == @name(ρq_tot)
            @. Yₜ.c.ρ += ᶜ∇ₕρD∇χₜ
        end
    end
end

# Run once to compile
horizontal_smagorinsky_lilly_tendency_mod!(Yt, Y, p, t, model)

e_mod = CUDA.@elapsed horizontal_smagorinsky_lilly_tendency_mod!(Yt, Y, p, t, model)
println("Elapsed time: $e_mod s")
println("Speedup: $(bl / e_mod)x")

Elapsed time: 0.000895296 s
Speedup: 1.1368576x


In [20]:
CUDA.@profile horizontal_smagorinsky_lilly_tendency_mod!(Yt, Y, p, t, model)

Profiler ran for 3.48 ms, capturing 532 events.

Host-side activity: calling CUDA APIs took 108.24 µs (3.11% of the trace)
┌──────────┬────────────┬───────┬────────────────────────────────────┬────────────────┐
│ Time (%) │ Total time │ Calls │ Time distribution                  │ Name           │
├──────────┼────────────┼───────┼────────────────────────────────────┼────────────────┤
│    2.00% │   69.86 µs │    12 │   5.82 µs ± 5.18   (  2.86 ‥ 21.7) │ cuLaunchKernel │
└──────────┴────────────┴───────┴────────────────────────────────────┴────────────────┘

Device-side activity: GPU was busy for 1.01 ms (28.98% of the trace)
┌──────────┬────────────┬───────┬─────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Time (%) │ Total time │ Calls │ Time distribution                   │ Name                                                                                         